# DAY 2 Data PreProcessing

In [2]:
from litellm import completion
from dotenv import load_dotenv
import json
from pricer.batch import Batch
from pricer.items import Item
import pandas as pd

load_dotenv(override=True)

True

In [3]:
LITE_MODE = True

In [4]:
username = "ed-donner"
dataset = f"{username}/items_raw_lite" if LITE_MODE else f"{username}/items_raw_full"
train,val,test = Item.from_hub(dataset)

items = train + val + test

print(f"Items lenght is {len(items):,} items")

print(items[0].id)

print(items[2])


Items lenght is 22,000 items
None
title='Bose QuietComfort 35 (Series I) Wireless Noise Cancelling Headphones - Silver (Renewed)' category='Electronics' price=240.0 full='Bose QuietComfort 35 (Series I) Wireless Noise Cancelling Headphones - Silver (Renewed)\n[\'Quiet Comfort 35 wireless headphones are engineered with world-class noise cancellation that makes quiet sound quieter and music sound better. Free yourself from wires and connect easily to your devices with Bluetooth and NFC pairing. Volume-optimized EQ gives you balanced audio performance at any volume, while a noise-rejecting dual microphone provides clear calls, even in windy or noisy environments. Voice prompts and intuitive controls make communicating and controlling your music hassle-free. A lithium-ion battery gives you up to 20 hours of wireless play time per charge. And if you anticipate a situation where charging may not be possible, just plug in the included audio cable. Wired mode gives you up to 40 hours of play t

In [5]:
for index,item in enumerate(items):
    item.id = index

print(f"Item 2 is: {items[2].id}")

Item 2 is: 2


In [6]:
SYSTEM_PROMPT= """Create a concise description of a product. Respond only in this format. Do not include part numbers.
Title: Rewritten short precise title
Category: eg Electronics
Brand: Brand name
Description: 1 sentence description
Details: 1 sentence on features"""

In [7]:
print(items[0].full)

Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)
['From the Manufacturer', "When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we're the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid"]
['Interior half only', 'Requires F58 to complete handle set', 'Non handed knob style', '4" minimum center to center door prep required for this two piece model.', 'Lifetime Mechanical and Finish Warranty']
{"Material": "Metal", "Brand": "", "Color": "Oil Rubbed Bronze", "Exterior Finish": "Bronze", "Special Feature": "Easy to Install", "Age Range (Description)": "Adult", "Included Components": "Deadbolt, Knob", "Item Weight": "1.5 pounds", "Handle Material": "Bronze", "Package Type": "Standard Packaging", "Unit Count": "1.0 Count", "Number of Items": "1", "Manufacturer": "Schlage", "Product Dimensions": "8.1 x 4

In [8]:
messages =[{"role":"system","content":SYSTEM_PROMPT},{"role":"user","content":items[0].full}]
response = completion(messages=messages,model="groq/openai/gpt-oss-20b",reasoning_effort="low")

print(response.choices[0].message.content)
print(f"input tokes : {response.usage.prompt_tokens}")
print(f"output tokes :{response.usage.completion_tokens}")
print(f"Cost: {response._hidden_params['response_cost']*100:.3f} cents")

Title: Schlage F59 Interior Knob with Deadbolt – Oil Rubbed Bronze  
Category: Hardware  
Brand: Schlage  
Description: An oil‑rubbed bronze interior knob set with a matching deadbolt for secure, easy installation.  
Details: Features a durable metal construction, lifetime mechanical and finish warranty, and requires a 4‑inch door prep for installation.
input tokes : 446
output tokes :91
Cost: 0.006 cents


In [9]:
messages =[{"role":"system","content":SYSTEM_PROMPT},{"role":"user","content":items[0].full}]
response = completion(messages=messages,model="ollama/llama3.2:latest",api_base="http://localhost:11434")

print(response.choices[0].message.content)
print(f"input tokes : {response.usage.prompt_tokens}")
print(f"output tokes :{response.usage.completion_tokens}")
print(f"Cost: {response._hidden_params['response_cost']*100:.3f} cents")

### Product:
Title: Schlage Front Door Security Handle Set
Category: Home Improvement
Brand: Schlage
Description: Provides a reliable and secure entrance to your home with its precision-engineered mechanism.
Details: Features easy installation, making it straightforward to install and maintain.
input tokes : 406
output tokes :56
Cost: 0.000 cents


In [10]:
MODEL = "gpt-4.1-mini"


In [11]:
def make_jsonl(item, customer_id=None):
    """customer_id overrides item.id (use loop index when writing JSONL to avoid None)."""
    cid = customer_id if customer_id is not None else item.id
    body = {"model":MODEL,"messages":[{"role":"system","content":SYSTEM_PROMPT},{"role":"user","content":item.full}]}
    line = {"custom_id": str(cid), "method": "POST", "url": "/v1/chat/completions", "body": body}
    return json.dumps(line)

In [12]:
make_jsonl(items[0])

'{"custom_id": "0", "method": "POST", "url": "/v1/chat/completions", "body": {"model": "gpt-4.1-mini", "messages": [{"role": "system", "content": "Create a concise description of a product. Respond only in this format. Do not include part numbers.\\nTitle: Rewritten short precise title\\nCategory: eg Electronics\\nBrand: Brand name\\nDescription: 1 sentence description\\nDetails: 1 sentence on features"}, {"role": "user", "content": "Schlage F59 AND 613 Andover Interior Knob with Deadbolt, Oil Rubbed Bronze (Interior Half Only)\\n[\'From the Manufacturer\', \\"When you have a Schlage handleset on your front door, you ensure your security as well as your peace of mind. After all, we\'re the leader in security devices, trusted for over 85 years. All Schlage handlesets are precision engineered, featuring 100% solid\\"]\\n[\'Interior half only\', \'Requires F58 to complete handle set\', \'Non handed knob style\', \'4\\" minimum center to center door prep required for this two piece model.\

In [155]:
items[0].summary

'Title: Schlage Andover Interior Knob with Deadbolt in Oil Rubbed Bronze  \nCategory: Home Security  \nBrand: Schlage  \nDescription: Durable interior half knob and deadbolt set designed for security and ease of installation.  \nDetails: Features a non-handed knob style, requires a compatible exterior set to complete, and comes with a lifetime mechanical and finish warranty.'

In [13]:

def make_file(start, end, filename):
    with open(filename, "w") as f:
        for i in range(start, end):
            f.write(make_jsonl(items[i], customer_id=i))
            f.write("\n")

In [14]:
make_file(0,1000,"jsonl/0_1000.jsonl")

In [15]:
import os
from openai import OpenAI
openai= OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [16]:
with open("jsonl/0_1000.jsonl","rb") as f:
    response =openai.files.create(file=f,purpose="batch")
response

FileObject(id='file-65k9Bux81HeWNAXzmJiRJ9', bytes=2198443, created_at=1769547147, filename='0_1000.jsonl', object='file', purpose='batch', status='processed', expires_at=1772139147, status_details=None)

In [17]:
file_id=response.id
file_id

'file-65k9Bux81HeWNAXzmJiRJ9'

In [18]:
response = openai.batches.create(completion_window="24h", endpoint="/v1/chat/completions", input_file_id=file_id)
response

Batch(id='batch_6979259c1c6881909abc42eb3e18461f', completion_window='24h', created_at=1769547164, endpoint='/v1/chat/completions', input_file_id='file-65k9Bux81HeWNAXzmJiRJ9', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1769633564, failed_at=None, finalizing_at=None, in_progress_at=None, metadata=None, model=None, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0), usage=BatchUsage(input_tokens=0, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=0, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=0))

In [19]:
import time

batch_id = response.id  # from batches.create
poll_interval = 60  # seconds
timeout_hours = 25  # stop if still not done (batch window is 24h)
start = time.time()

while True:
    result = openai.batches.retrieve(batch_id=batch_id)
    elapsed = (time.time() - start) / 60
    print(f"[{elapsed:.0f} min] Batch status: {result.status}", flush=True)
    if result.status == "completed":
        print(f"Output file ID: {result.output_file_id}", flush=True)
        break
    if result.status in ("failed", "expired", "cancelled"):
        print(f"Batch ended: {result.status}. Errors: {result.errors}", flush=True)
        result = None
        break
    if elapsed >= timeout_hours * 60:
        print("Timeout waiting for batch.", flush=True)
        result = None
        break
    time.sleep(poll_interval)

[0 min] Batch status: in_progress
[1 min] Batch status: in_progress
[2 min] Batch status: in_progress
[3 min] Batch status: finalizing
[4 min] Batch status: finalizing
[5 min] Batch status: completed
Output file ID: file-2saUmyvsqk3MkvY73kuvKK


In [ ]:
response=openai.batches.retrieve(batch_id='batch_6979259c1c6881909abc42eb3e18461f')

print(f"response status is {response.output_file_id}")


response status is file-YcDnpCDsF59s3S27sGx6ah


In [151]:
if result and result.output_file_id:
    content = openai.files.content(result.output_file_id)
    content.write_to_file("jsonl/batch_results.jsonl")
    print("Saved to jsonl/batch_results.jsonl")
else:
    print("No output file — batch not completed or failed. Run the cell above and wait for completion.")

Saved to jsonl/batch_results.jsonl


In [152]:
with open("jsonl/batch_results.jsonl","r") as f:
    for line in f:
        json_line = json.loads(line)
        id = int(json_line["custom_id"])
        summary = json_line["response"]["body"]["choices"][0]["message"]["content"]
        items[id].summary = summary

In [1]:
print(items[0].full)

NameError: name 'items' is not defined

In [164]:
Batch.create(items, LITE_MODE)

Created 88 batches


In [163]:
Batch.run()

  0%|          | 0/66 [00:00<?, ?it/s]

PermissionDeniedError: Error code: 403 - {'error': {'message': 'Not available for your plan', 'type': 'permissions_error', 'code': 'not_available_for_plan'}}